In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
from pycocotools.coco import COCO
site = 'val'
img_dir = f'./data/jinan/pan/{site}/JPEGImages'
json_path = f'./data/jinan/fusion/{site}/{site}.json'

In [2]:
coco = COCO(json_path)
img_ids = set(_['image_id'] for _ in coco.anns.values())

loading annotations into memory...
Done (t=0.41s)
creating index...
index created!


In [3]:
from mmdet.datasets.pipelines import Compose
from mmcv.parallel import collate, scatter
from mmdet.datasets import replace_ImageToTensor
def inference_detector(model, imgs):
    """Inference image(s) with the detector.

    Args:
        model (nn.Module): The loaded detector.
        imgs (str/ndarray or list[str/ndarray] or tuple[str/ndarray]):
           Either image files or loaded images.

    Returns:
        If imgs is a list or tuple, the same length list type results
        will be returned, otherwise return the detection results directly.
    """

    if isinstance(imgs, (list, tuple)):
        is_batch = True
    else:
        imgs = [imgs]
        is_batch = False

    cfg = model.cfg
    device = next(model.parameters()).device  # model device

    if isinstance(imgs[0], np.ndarray):
        cfg = cfg.copy()
        # set loading pipeline type
        cfg.data.test.pipeline[0].type = 'LoadImageFromWebcam'

    cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
    test_pipeline = Compose(cfg.data.test.pipeline)

    datas = []
    for img in imgs:
        # prepare data
        if isinstance(img, np.ndarray):
            # directly add img
            data = dict(img=img)
        else:
            # add information into dict
            data = dict(img_info=dict(filename=img), img_prefix=None)
        # build the data pipeline
        data = test_pipeline(data)
        datas.append(data)

    data = collate(datas, samples_per_gpu=len(imgs))
    # just get the actual data from DataContainer
    data['img_metas'] = [img_metas.data[0] for img_metas in data['img_metas']]
    data['img'] = [img.data[0] for img in data['img']]
    if next(model.parameters()).is_cuda:
        # scatter to specified GPU
        data = scatter(data, [device])[0]
    else:
        for m in model.modules():
            assert not isinstance(
                m, RoIPool
            ), 'CPU inference with RoIPool is not supported currently.'

    # forward the model
    with torch.no_grad():
        results = model(return_loss=False, rescale=True, **data)

    if not is_batch:
        return results[0]
    else:
        return results

In [4]:
from mmdet.apis import init_detector, show_result_pyplot
import mmcv
import glob
work_dir = './venus_last_tf/15'
config_file = glob.glob(os.path.join(work_dir, '*.py'))[0]
checkpoint_file = os.path.join(work_dir, 'latest.pth')
cfg_options = {}
cfg_options = {'model.roi_head.polygon_head.polyrnn_head.weight_kernel_params.kernel_size': 3, 
               'model.roi_head.polygon_head.polyrnn_head.weight_kernel_params.type': 'gaussian'}

model = init_detector(config_file, checkpoint_file, device='cuda:0', cfg_options=cfg_options)

Use load_from_local loader


In [5]:
import numpy as np
import mmcv
import cv2
import torch
import tifffile as tiff
def show_result(img_path, result, score_thr):
    img = tiff.imread(img_path)
    img = img[:, :, :3][:, :, ::-1]
    img = img.copy()
    bbox_result, segm_result = result
    bboxes = np.vstack(bbox_result)
    labels = [
        np.full(bbox.shape[0], i, dtype=np.int32)
        for i, bbox in enumerate(bbox_result)
    ]
    labels = np.concatenate(labels)
    # draw segmentation masks
    segms = None
    if segm_result is not None and len(labels) > 0:  # non empty
        segms = mmcv.concat_list(segm_result)
        if isinstance(segms[0], torch.Tensor):
            segms = torch.stack(segms, dim=0).detach().cpu().numpy()
        else:
            segms = np.stack(segms, axis=0)
    scores = bboxes[:, -1]
    inds = scores > score_thr
    bboxes = bboxes[inds, :]
    labels = labels[inds]
    if segms is not None:
        segms = segms[inds, ...]
    for i, (bbox, label) in enumerate(zip(bboxes, labels)):
        mask = segms[i].astype(np.uint8)
        contours, hierarchy = cv2.findContours(mask[..., None], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(img, contours, -1, (0,0,255), 2)
    return img

In [6]:
print(len(img_ids))
output_dir = './polyrnn_visual_jinan'
os.makedirs(output_dir, exist_ok=True)

799


In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
img_ids = list(img_ids)

for bi, img_id in enumerate(tqdm(img_ids)):
    fn = coco.load_imgs([img_id])[0]['file_name']
    img_path = os.path.join(img_dir, fn)
    
    result = inference_detector(model, img_path)
#     show_result_pyplot(model, img_path, result, score_thr=0.2)
    img_path = img_path.replace('pan', 'fusion')
    img = show_result(img_path, result, score_thr=0.6)
#     img = img[:, :, ::-1]
#     plt.figure(figsize=(15, 10))
#     plt.axis('off')
#     plt.imshow(img)
#     plt.show()
    out_path = os.path.join(output_dir, fn)
    out_path = out_path.replace('.tif', '.png')
    cv2.imwrite(out_path, img)
#     if bi > 10:
#         break

 98%|█████████▊| 784/799 [1:08:03<01:29,  5.98s/it]